<img src=banner.png>

# <a name="0">Measuring and Mitigating Toxicity in Large Language Models</a>

Building and operating machine learning applications responsibly requires an active, consistent approach to prevent, assess, and mitigate harm. This workshop guides you through how to identify toxicity in LLM generated summaries and how to mitigate and reduce toxicity.

In this workshop you will:
1. <a href="#1">Load a dataset</a>
2. <a href="#2">Load and use a Large Language Model (LLM)</a>
3. <a href="#3">Evaluate LLM generated summaries for toxicity</a>
4. <a href="#4">Mitigate toxicity using guardrails</a>
5. <a href="#5">Reduce toxicity using a Direct Optimization Policy (DPO)</a>


**Learning Objectives**

In this workshop you will learn to:

- Use a LLM for text summarization
- Apply toxicity metrics to evaluate summaries
- Use various toxicity classifiers and libraries
- Define guardrails to mitigate toxicity
- Tune a model to mitigate toxicity

**Runtime**

This notebook takes about 90 minutes to complete (using some inbuilt shortcuts).

### Let's get started

Start by upgrading [pip](https://pypi.org/project/pip/) (a Python package management system) and install all required libraries from the provided requirements.txt file.

In [2]:
!pip install -q -U pip --root-user-action=ignore
!pip3 install -q -r requirements.txt --root-user-action=ignore
!python3 -m spacy download en_core_web_sm

In [3]:
import warnings

warnings.filterwarnings(
    action="ignore",
    category=UserWarning,
)

import transformers, torch

transformers.logging.set_verbosity_error()

from tqdm.auto import tqdm as notebook_tqdm

## <a name="1">1. Load a dataset</a>
(<a href="#0">Go to top</a>)

In this notebook, you will be working with the "[Cornell Movie-Dialogs Corpus](https://convokit.cornell.edu/documentation/movie.html)", a large metadata-rich collection of fictional conversations extracted from raw movie scripts. The dataset contains 220,579 conversational exchanges between 10,292 pairs of movie characters in 617 movies.


In [4]:
from utils.data_utils import _prepare_data

# load the data
movie_df = _prepare_data()

# show the data
movie_df.head(2)

,movie,dialogue,genre
0,"""murderland""","Jesus, my legs are asleep. I'll never be able ...",crime
1,10 things i hate about you,They do not! They do to! I hope so. She okay? ...,comedy


### 1.1. Exploring the data
Have a look at some examples text snippets for different genres (action and comedy) in the dataset. You can execute this cell multiple times as it will return different examples every time.

In [5]:
from utils.data_utils import _explore_df

_explore_df(movie_df)   

Action: ...you keep such a firm track of the dead. My department doesn't have any record of that, Mrs. Bickerman. Oh, my husband passed away almost two years ago. <u>I'm</u> <u>trying</u>!!! Go!!! Jesus.  About twelve more yards.  Keep coming just like that. <u>No</u>.  If you don't kill him instantly, I'm dead and you'll only kill him instantly if you get his brain, which is about the size of a cherry.  And even if you <u>were</u> on target, a bullet might not penetrate his hide. I might get a shot. He's ...

Comedy: ...p it. Joe - Josephine - help! The one-legged jockey said - 'Don't worry about me, baby.  I ride side-saddle.' What did he say? - so the one-legged jockey said - Manhattans?  This time of night? Okay. It's private.  Go away. This a private clambake, or can anybody join? Where were you guys? Me and Tiny, we had them cornered - but we lost 'em in the shuffle. What happened? They wouldn't be caught dead in Chicago - so we'll finish the job here.  Come on. Those two musicia

<div class="alert alert-block alert-warning">
<b>Exercise 1</b>: Did you make any noteworthy observations? Is one genre maybe using more explicit language compared to another genre?
</div>

### 1.2. HuggingFace Datasets 
LLMs require the data to be stored in a specific format; use the [HuggingFace 🤗 Datasets](https://huggingface.co/docs/datasets/index) library to convert the dataframe.

In [6]:
from datasets import Dataset

# convert the data
movie_dataset = Dataset.from_pandas(movie_df)

# show the data
movie_dataset

Dataset({
    features: ['movie', 'dialogue', 'genre'],
    num_rows: 617
})

You can see that there are 617 distinct movies, to move through the remainder of the notebook more quickly, select the first 200 samples.

In [7]:
# select a sample of 200
dataset = movie_dataset.select(range(200))

# save the dataset to disk
dataset.save_to_disk("movie_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

Now that the dataset is in this new format, it is very easy to use LLM helper libraries to evaluate the toxicity for the input text. The `_add_toxicity_column` method will add a new column to the dataset containing a toxicity score.

In [8]:
from utils.eval_utils import _add_toxicty_column

# pass the "dialogue" column through the toxicity evaluator
dataset = _add_toxicty_column(dataset, "dialogue")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

In [9]:
import numpy as np

print(np.mean(dataset.filter(lambda example: example["genre"]=="action")["toxicity_score"]))
print(np.mean(dataset.filter(lambda example: example["genre"]=="comedy")["toxicity_score"]))

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

0.1645884431744217


Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

0.10107923420813367


As you can see, the crime genre text is showing higher toxicity compared to comedy.

<div class="alert alert-block alert-warning">
<b>Exercise 2</b>: Calculate the toxicity for another genre.
</div>

In [10]:
##### complete your code here #####


###################################

Delete all old variables that are no longer needed to free up memory with `del`.

In [11]:
del movie_dataset, movie_df, dataset

Make sure to release the memory after deleting the objects and variables that are no longer in use.

In [12]:
import gc

gc.collect()

90732

<div class="alert alert-block alert-success">
<b>Conclusion</b>: In this section, you loaded a movie transcript dataset and converted it into a HuggingFace Dataset.
</div>

## <a name="2">2. Load and use a Large Language Model</a>
(<a href="#0">Go to top</a>)

[T5 (Text-To-Text Transfer Transformer)](https://github.com/google-research/text-to-text-transfer-transformer) is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks. T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, including machine translation, **document summarization**, question answering, and classification tasks (e.g., sentiment analysis). 

<div style="text-align: center;">
<img src="https://camo.githubusercontent.com/623b4dea0b653f2ad3f36c71ebfe749a677ac0a1/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f6d61782f343030362f312a44304a31674e51663876727255704b657944387750412e706e67" width="700"/>
</div>

For more details have a look at the T5 documentation on HuggingFace 🤗 [here](https://huggingface.co/docs/transformers/model_doc/t5).

### 2.1. Loading T5

First, you have to download the T5 model using the `T5ForConditionalGeneration` class provided by the [HuggingFace 🤗 transformers library](https://github.com/huggingface/transformers) as well as the corresponding tokenizer `T5Tokenizer`. You can think of tokens as pieces of words that are required to pass information to LLMs. 

In [13]:
from transformers import T5ForConditionalGeneration

Load the model and use the GPU as preferred device type.

In [14]:
import torch

# load the model
model_t5 = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-large",
    device_map={"": 0},  # this will load the model in GPU
    torch_dtype=torch.float32,
    return_dict=True,
)

Together with a tokenizer the model can be used to generate text. 
For English, **1 token is approximately 4 characters or 0.75 words**. This will be important to consider as LLMs are limited by the number of tokens they can pay attention to per prompt. Go ahead and initialize a tokenizer next.

In [15]:
from transformers import AutoTokenizer

# load the tokenizer
tokenizer_t5 = AutoTokenizer.from_pretrained(
    "google/flan-t5-large",
    skip_special_tokens=True,
    return_tensors="pt",
    truncation=True,
    use_fast=True,
)

Load the dataset that you previously created with `load_from_disk`.

In [16]:
from datasets import load_from_disk

# specify the dataset to load
dataset = load_from_disk("movie_dataset")

Let's create a prompt by joining an instruction to summarize text with the actual movie dialogue:

In [17]:
# create a prompt and use an example dialogue
inference_prompt = (
    "Summarize the following conversation from a movie script:  \n\n'''%s'''"
    % dataset[0]["dialogue"]
)

# let's look at the prompt but shorten the output to reduce the amount of text
print(inference_prompt[:235])

Summarize the following conversation from a movie script:  

'''Jesus, my legs are asleep. I'll never be able to win this shit. You must come in first place to move on ! Pick with expediency ! Great.. great. Is there any kind of hint o


Have a look at what this looks like when converted to tokens:

In [18]:
print(tokenizer_t5(inference_prompt[:235]).input_ids)

[12198, 1635, 1737, 8, 826, 3634, 45, 3, 9, 1974, 4943, 10, 3, 31, 31, 31, 7851, 7, 302, 6, 82, 6217, 33, 17915, 5, 27, 31, 195, 470, 36, 3, 179, 12, 1369, 48, 3, 7, 10536, 5, 148, 398, 369, 16, 166, 286, 12, 888, 30, 3, 55, 8356, 28, 22879, 4392, 3, 55, 1651, 5, 5, 248, 5, 27, 7, 132, 136, 773, 13, 12037, 3, 32, 1]


The **number of tokens passed to an LLM through the tokenizer should not be greater than the number of tokens used in pre-training**. T5 was pre-trained using 512 input tokens, and with `truncation=True` all text beyond 512 tokens will be truncated.

## 2.2. Using T5 for inference on individual movie examples


To generate a summary with T5 you need an inference pipeline that encodes the input (tokenization), passes the tokens through the model and then decodes everything back to text.

In [19]:
from utils.model_utils import _generate_summary, _format_llm_output

Try the inference pipeline. The pipeline will return a list that you have to access to retrieve the LLM generated output.

In [20]:
# pass the prompt to the pipeline and apply formatting
_format_llm_output(_generate_summary(inference_prompt, model_t5, tokenizer_t5))

<div class="alert alert-block alert-info"><pad> Bruce and Johnny are at a roller coaster park. Bruce wants to win the race, but he's not very good at it. He has an embezzler and his accomplice. Johnny is an investment banker. He sees blood bathes everyday. No one even batted an eyelash.</s></div>

This summary looks okay but important characters that appear in the dialogue are not mentioned at all. This is due to the limited number of tokens T5 can 'keep track of'. Later, you will see a method that can help fix this issue.

<div class="alert alert-block alert-warning">
<b>Exercise 3</b>: Recreate the example above but for another movie.
</div>

In [21]:
##### complete your code here #####


###################################

# Try to find toxic one (swear word)

In [22]:

_format_llm_output(_generate_summary(dataset[15]["dialogue"], model_t5, tokenizer_t5))


<div class="alert alert-block alert-info"><pad> Landon, you're a good kid. You'll be fine. I love you. Good-bye. Bye bye. And kisses to you and your mommy for now. See you tomorrow. Love you too.</s></div>

Before proceeding, delete the prompts that were used for inference; e.g. <code>del inference_prompt</code> and also clear the instance memory with <code>gc.collect()</code>. 

In [23]:
del inference_prompt

In [24]:
import gc

gc.collect()

33

## 2.2. Using T5 for inference on all movie examples

The goal of this section is to summarize all movie dialogues. As previously mentioned, there is one very important caveat though - **Large Language Models are only able to pay attention to a limited number of tokens**. The amount of tokens an LLM can understand is called context window. 

In [25]:
model_t5.config.__dict__["n_positions"]

512

As shown above, the context window for T5 models is 512 tokens. This means the movie transcript needs to be split into chunks of this lenght and summarised one by one. Then, a final summary needs to be created.

<div style="text-align: center;">
<img src="map_chain.png" width="900"/>
</div>

### 2.2.1. Chunking the movie transcripts
Let's start by creating chunks of the movie transcripts. One simple way to create chunks of text is to write a helper function and then apply this helper function to all the movies in the dataset.

In [26]:
def create_chunks(sample, CHUNK_LENGTH):
    """
    Splits a given text into chunks of a specified length and adds metadata to each chunk.
    """
    chunks = []
    # loop over entire text in steps of chunk size
    for c, i in enumerate(range(0, len(sample["dialogue"]), CHUNK_LENGTH)):
        # extract text
        chunk_text = sample["dialogue"][i : i + CHUNK_LENGTH]
        # create dictionary with the chunked text and metadata
        chunks.append(
            # remove uncompleted sentences with string split
            {
                "text": ".".join(chunk_text.split(".")[1:-1]).lstrip(),
                "metadata": {"page": c, "num_words": len(chunk_text)},
            }
        )
    # create new column
    sample["chunks"] = chunks
    return sample

Create the chunks for all the movie transcripts in the dataset with the help of `.map()`; this method efficiently applies the `create_chunks` function to all datapoints. Whenever you have additional parameters to pass to the model, you need to use a helper method, such as `partial`.

In [27]:
from functools import partial

# use partial to pass the arguments to the map function
dataset = dataset.map(partial(create_chunks, CHUNK_LENGTH=1650), batched=False)

<div class="alert alert-block alert-warning">
<b>Exercise 4</b>: Think about how the chunking could be improved. Hint: Look for text splitters in the LangChain documentation.
</div>

In [28]:
###### write down ideas here ######


###################################

### 2.2.2. Prepare prompt templates and pipeline
Now that the transcripts are chunked, let's start by setting up a prompt template for the intermediate (chunk) summaries. A prompt template is special construct that can parse input variables. Prompt templates can be applied to all the items in a dataset and help with consistency and reproducability. 

**Prompt templates**: Prompt templates can be very elaborate as ultimately the prompt is the only input the LLM sees - the better the prompt, the better the result. In the case of T5, the prompts used for pre-training all used _summarize:_, so this is what you should use.

In [29]:
from langchain import PromptTemplate

Define the prompt template for the movie chunks.

In [30]:
# prompt template for movie chunks
chunk_template = """summarize the movie dialogue chunks: ```{text}``` \n\n"""
chunk_prompt = PromptTemplate(
    template=chunk_template, input_variables=["text"]
)

# prompt template for final summary
combine_template = """summarize these snippets of text: ```{text}``` \n\n"""
combine_prompt = PromptTemplate(
    template=combine_template, input_variables=["text"]
)

**Pipelines**: To get a summary from the model, use 🤗 HuggingFace [pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) together with 🦜️🔗 LangChain's wrapper [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html). Pipelines are a great and easy way to use models for inference that offer a simple API dedicated to several tasks (e.g. [`summarization`](https://huggingface.co/transformers/v3.0.2/task_summary.html#summarization)).

In [31]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

pipe = HuggingFacePipeline(pipeline = pipeline(
            task="summarization",
            model=model_t5,
            tokenizer=tokenizer_t5,
            num_return_sequences=1,
            do_sample=False,
            early_stopping=True,
            num_beams=2,
            min_length=65,
            max_length=350,
            repetition_penalty=2.0,
            )
        )



### 2.2.3. Create summaries of chunks and final summary

At this point now, you could apply the prompt template to all the chunks of movie transcripts to obtain your summaries, combine them back together and create a final summary. This would be a very lengthy and error-prone process, so instead make use of an increasingly popular toolkit: [🦜️🔗 LangChain](https://python.langchain.com/docs/get_started/introduction); in particular the [`load_summarize_chain`]((https://python.langchain.com/docs/use_cases/summarization)). This **chain will take the chunks, summarize them and then pass all the summaries to the LLM to create the final summary**.

In [32]:
from langchain.chains.summarize import load_summarize_chain

map_reduce_chain = load_summarize_chain(
    llm=pipe,
    chain_type="map_reduce",
    map_prompt=chunk_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=False,
)

There is one more small caveat: LangChain expects all text to be passed as `Document` type following the 🦜️🔗 LangChain schema. So you will have to convert the chunks to the expected schema. Then you can test the summarization chain:

In [33]:
from langchain.schema import Document

sample_doc = [
    Document(page_content=split["text"], metadata=split["metadata"])
    for split in dataset[0]["chunks"]
]

# turn on verbosity for chain
map_reduce_chain.llm_chain.verbose = True

# run the summarization chain
map_reduce_example = map_reduce_chain({"input_documents": sample_doc})

# show the result
_format_llm_output(map_reduce_example["output_text"])



> Entering new LLMChain chain...
Prompt after formatting:
summarize the movie dialogue chunks: ```I'll never be able to win this shit. You must come in first place to move on ! Pick with expediency ! Great.. great. Is there any kind of hint or anything ? So many tubes to get in, only one is the way out. Great ! You couldn't be a fan of Dolphins or some other less threatening animal. And now ladies and gentlemen it's time for the fabulous CROCODILE SHOW ! Where's the dock ? That's it folks for the Demon Boat Ride.... Everybody off. Watch your hands and feet. Good. Nice to see you, Bruce. Johnny. I'll do the standard research and have them in by midnight, pending any unforseen problems. I have an embezzler and his accomplice. Good morning. What do you have today ? No one has a gun to your head. I'm an investment banker. I see blood bathes everyday. Besides, mine is not to question why, min is but to do or die. Doesn't it bother you to see this kind of brutal death ? I mean , I can unde

<div class="alert alert-block alert-info">The movie opens with a man (Jackson) talking to a woman (McCarthy) in a restaurant. They are talking about their jobs and how they feel about their current situation. The woman says that she has been fired from her job because she is not satisfied with the pay. The man says that he was fired because he doesn't like his job. The woman says that he was fired because he doesn't like his boss.</div>

<div class="alert alert-block alert-warning">
<b>Exercise 5</b>: Recreate the example above but for another movie.
</div>

In [34]:
##### complete your code here #####


###################################

The next step is to generate all summaries. Because the LLM has to generate summaries for every chunk of text, as well as a final summary, the time to create summaries for all movies in the dataset is approximately 6 hours. You can find the code below, but please skip this code cell and simply load the pre-generated summaries.


```
from utils.model_utils import _add_summaries

# create summaries
summaries_dataset = dataset.map(
    partial(
        _add_summaries,
        chain=map_reduce_chain,
    ),
    batched=False,
)
````

If you need to load in the dataset, you can do so with `load_from_disk('summaries_dataset')`. Make sure to import the method first with `from datasets import load_from_disk`.

<div class="alert alert-block alert-success">
<b>Conclusion</b>: At this point, you have summaries for all the movies and it is time to check whether those summaries contain any hate speech, slurs or toxic remarks. Generally, you expect the toxicity values in a summarization task to be low unless the text being summarised itself already contains toxic speech.
</div>

In [35]:
del map_reduce_chain, pipe, dataset
gc.collect()
torch.cuda.empty_cache()

# 3. Evaluate LLM generated summaries for toxicity

To evaluate toxicity you can load the 🤗 [evaluate](https://huggingface.co/docs/evaluate/index) library and initialize a toxicity evaluator object. The model that will be used to evaluate toxicity is the [RoBERTa](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) model. Start by loading in the data.

In [36]:
from datasets import load_from_disk

summaries_dataset = load_from_disk("summaries_dataset")

To evaluate the movie summary for toxicity, simply pass a list containing the summary text to the toxicity evaluator object. The aggregation parameter options are `None`, `maximum` and `ratio`.

In [37]:
from utils.eval_utils import _evaluate_toxicity

_evaluate_toxicity([summaries_dataset[0]["summary"]], aggregation_method=None)

[0.7981389760971069]

In [38]:
string= 'This is the epic story of Christopher Columbus, a renegade explorer determined to prove the world is round. Facing ridicule and rejection, Columbus overcomes countless obstacles to gain support for his bold expedition. When his crew threatens mutiny, outnumbered and lost in uncharted waters, Columbus must confront the ultimate test of faith and leadership. Against all odds, he discovers the New World and claims it for Spain. But as Governor of the new colonies, Columbus faces political intrigue, treachery and bloody conflict with the native tribes. Accused of tyranny, he falls from grace and dies penniless, never knowing the true magnitude of his achievement. Columbus is a flawed but visionary hero who gambles everything on his daring quest into the unknown. With vivid historical detail, the movie captures the danger and drama of his groundbreaking voyages that opened up the globe. This is an epic adventure about faith, courage and destiny that changed the world forever.'

_evaluate_toxicity([string])

[0.0008151092915795743]

<div class="alert alert-block alert-warning">
<b>Exercise 6</b>: Calculate the max toxicity score for <code>summaries_dataset[1]["summary"]</code>. In addition to <code>aggregation_method = None</code> also try <code>aggregation_method = "maximum"</code>. You can also check out the summary text itself.
</div>

In [39]:
##### complete your code here #####


###################################

Now that you evaluated for a few movies manually, it is time to evaluate all movie summaries and obtain a list of toxicity scores.

In [40]:
from utils.eval_utils import _add_toxicty_column

summaries_dataset = _add_toxicty_column(summaries_dataset, "summary")

Now that you have the toxicity scores per movie, let's have a look at the toxicity across all summaries.

In [41]:
import numpy as np

np.mean(summaries_dataset["toxicity_score"]), np.std(summaries_dataset["toxicity_score"])

(0.2025899624430167, 0.31525101796310845)

<div class="alert alert-block alert-warning">
<b>Exercise 7</b>: Try to calculate mean toxicity for two different movie genres: comedy and crime.
</div>

In [42]:
##### complete your code here #####


###################################

<div class="alert alert-block alert-success">
<b>Conclusion</b>: We have seen that some summaries are toxic and would like to remediate this. The first option to mitigate toxicity would be to use a protective wrapper around the LLM itself. This is called a guardrail and is a very useful technique to employ whenever you don't have access to the model itself or not sufficient time or compute resources to make any modifications to the LLM. If however, you are interested in modifying the LLM, you can use techniques such as fine-tuning or reinforcement learning through human feedback.
</div>

In [43]:
del summaries_dataset, pipeline, load_summarize_chain, model_t5, tokenizer_t5
gc.collect()
torch.cuda.empty_cache()

# 4. Mitigate toxicity using Guardrails

In this section you will explore coding examples for so-called guardrails that can filter certain keywords or that leverage metrics to decide if content is harmful. To get started with [Guardrails.ai](https://docs.guardrailsai.com/) you need a `Validator` and a `RAIL spec` (Reliable AI markup Language spec).

A Validator is a class that contains a `validate` method. The validation could be anything you can possibly think of. You could check whether values are in a certain range or check for keywords as the example below shows. With the validation, it is also possible to define a corrective action to take, such as obfuscating the problematic parts or refusing to create an output altogether. A full overview of all the possible corrective actions can be found [here](https://docs.guardrailsai.com/concepts/output/#specifying-corrective-actions).


## 4.1. Guardrails to obfuscate unwanted words
Have a look at the below guardrail which filters for a pre-defined keyword list.

In [44]:
from guardrails.validators import *
from typing import Dict, Any

# provide a name for the validator to use in the RAIL spec later
@register_validator(name="is-keyword-free", data_type="string")
class IsKeywordFree(Validator):
    # the Validator class needs to contain a validate method
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        # set up a list of words to filter for
        kw_list = ["vampire"]
        # check for forbidden words
        if any(kw in value for kw in kw_list):
            # replace forbidden words in output with ***
            for kw in kw_list:
                censored_text = value.replace(kw, "***")
            # display error message and return the fix value
            return FailResult(
                error_message=f"Expression '{value}' contains forbidden keyword.",
                fix_value=censored_text,
            )
        # else return pass
        return PassResult()

Once you have a validator, you need to create a RAIL spec. This is basically a file written in XML. In this RAIL spec, you need to specify the validator you want to use and create a placeholder for the prompt to pass through. 

In [45]:
rail_str = """
<rail version="0.1">

<output>
    <string
        name="summarize_statement"
        format="is-keyword-free"
        on-fail-is-keyword-free="fix"
    />
</output>

<prompt>
summarize:
${statement_to_be_summarized}
</prompt>

</rail>
"""

Next, everything needs to be merged together and passed to a Guard object.

In [46]:
import guardrails as gd

# create a Guard object from the above RAIL string
guard = gd.Guard.from_rail_string(rail_str)

Load the movie dataset from disk - in the next code cell you will pass the data to the T5 model.

In [47]:
from datasets import load_from_disk

movie_dataset = load_from_disk("movie_dataset")

Finally, pass the movie dialogue you want summarized and checked with the guardrail to the Guard object.

In [48]:
!nvidia-smi

Wed Nov 15 02:45:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   34C    P0    27W /  70W |    755MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [49]:
from utils.model_utils import _my_llm_api

# provide API to Guard and the prompt input
raw_llm_response, validated_response = guard(
    llm_api=_my_llm_api,
    prompt_params={"statement_to_be_summarized": movie_dataset[0]["dialogue"]},
)

# show the output
print(f"Validated Output: {validated_response}")

Validated Output: {'summarize_statement': "summary: Interview with the CEO of First Bank in New York City. He is impressed with his knowledge of the market and his understanding of the company's culture. He has been with the company for four years. He came from the Capitol merger in Chicago six months ago. He is equally impressed with his knowledge of the market and his understanding of the company's culture. The stock opened at a high price, but he thinks it will pick up."}


<div class="alert alert-block alert-warning">
<b>Exercise 8</b>: Create another validated response using guardrails.
</div>

In [50]:
##### complete your code here #####


###################################

## 4.2. Guardrails to filter profanties
Have a look at the below guardrail which filters profantities using a profanity classifier. Instead of using a pre-defined keyword list, you can also call a model that determines if a word is considered a profanity and filter it out.

In [51]:
from profanity_check import predict

@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        prediction = predict([value])
        if prediction[0] == 1:
            return FailResult(
                error_message=f"The result contains profanity and will be filtered.",
                fix_value="",
            )
        return PassResult()

Define the rail string again (make sure to use the correct validator name).

In [52]:
rail_str = """
<rail version="0.1">

<output>
    <string
        name="summarize_statement"
        format="is-profanity-free"
        on-fail-is-profanity-free="filter"
    />
</output>

<prompt>
summarize:
${statement_to_be_summarized}
</prompt>

</rail>
"""

Instantiate the Guard object.

In [53]:
# create a Guard object from the above RAIL string
guard = gd.Guard.from_rail_string(rail_str)

Pass a prompt to summraize through the Guard object.

In [54]:
# test string of profanities from Reddit
test_string = "I do like being a weirdo and a fucking asshole, so I'm glad your loser self has decided to fucking stop being a level-headed sissy, finally grew a fuckin pair of balls and asked! I can fucking cuss up a damn storm that'll make little pansies cry their faggoty bitch ass out! Seriously though, are you such a retarded idiot that you can't fucking figure this shit out? But to fucking answer your fucking question, profanity is what I'm fucking doing right now."

# provide API to Guard and the prompt input
raw_llm_response, validated_response = guard(
    llm_api=_my_llm_api,
    prompt_params={"statement_to_be_summarized": test_string},
)

# show the output
print(f"Validated Output: {validated_response}")

Validated Output: {}


Guardrails also provides a visual overview of what the prompt, raw LLM output and validated output look like.

In [55]:
guard.state.most_recent_call.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ summarize:                                                                                              │ │
    │ │ I do like being a weirdo and a fucking asshole, so I'm glad your loser self has decided to fucking stop │ │
    │ │ being a level-headed sissy, finally grew a fuckin pair of balls and asked! I can fucking cuss up a damn │ │
    │ │ storm that'll make little pansies cry their faggoty bitch ass out! Seriously though, are you such a     │ │
    │ │ retarded idiot that you can't fucking figure this shit out? But to fucking answer your fucking          │ │
    │ │ question, profanity is what I'm fucking doing right now.                                                │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━┓                                                                                      │ │
    │ │ ┃ Role ┃ Content ┃                                                                                      │ │
    │ │ ┡━━━━━━╇━━━━━━━━━┩                                                                                      │ │
    │ │ └──────┴─────────┘                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ {"summarize_statement": "I do like being a weirdo and a fucking asshole, so I'm glad your loser self    │ │
    │ │ has decided to fucking stop being a level-headed sissy, finally grew a fuckin pair of balls and asked!  │ │
    │ │ I can fucking cuss up a damn storm that'll make little pansies cry their faggoty bitch ass out!         │ │
    │ │ Seriously though, are you such a retarded idiot that you can't fucking figure this shit out? But to     │ │
    │ │ fucking answer the fucking question, profanity is what I'm "}                                           │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ {}                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In this example, it becomes very obvious that T5 is not actually able to summarize the input text as it clearly lacks the vocabulary and understanding. Thanks to the guardrail, the validated output is also empty.

<div class="alert alert-block alert-success">
<b>Conclusion</b>: You have seen guardrails as very effective and lightweight method to mitigate toxic outputs by adding a validation layer around the call to the LLM. Guardrails should be used whenever you are looking for a solution that does not require retraining the LLM itself.
</div>

# 5. (Optional): Mitigate toxicity using a Direct Optimization Policy (DPO)

The idea behind Direct Policy Optimization (DPO) is to provide human annotators with different outputs that were generated using a certain prompt. The human annotators will be tasked to simply indicate which output they prefer and which one they would like to reject. The preferred output, together with the rejected output and the prompt that was used can be use in a direct optimization approach. 

To use DPO for a model, three main steps are required:
1. create a dataset that includes 'prompt, preferred, rejected'
2. fine-tune the model on the dataset to ensure the vocabulary is in-distribution
3. train the model using the DPO algorithm

This section will consume a lot of device memory so it will be best to restart the kernel and start fresh.

In [56]:
from datasets import load_from_disk

movie_dataset = load_from_disk("movie_dataset")
summaries_dataset = load_from_disk("summaries_dataset")

## 5.1. Create DPO dataset

In [57]:
from functools import partial
from utils.data_utils import _return_prompt_and_responses

BATCH_DATA = 5

# reshape the dataset to format DPO expects
dpo_ds = summaries_dataset.map(
    partial(_return_prompt_and_responses, batch_multiplier=BATCH_DATA),
    batched=True,
    batch_size=BATCH_DATA,
    remove_columns=summaries_dataset.column_names,
)

# create train/eval split for fine-tuning
ds = summaries_dataset.train_test_split(train_size=150, test_size=50, seed=0)

## 5.2. Fine-tune model

In [58]:
from transformers import BitsAndBytesConfig, T5ForConditionalGeneration, TrainingArguments
from peft import LoraConfig, TaskType
import torch
from trl import SFTTrainer

# config to load base model in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# set up base model - T5 Large but with quantization config
model_t5_qn = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-large",
    quantization_config=bnb_config,
    device_map={"": 0},
)

# turn of cache to use updated model params
model_t5_qn.config.use_cache = False

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "google/flan-t5-large",
    skip_special_tokens=True,
    return_tensors="pt",
    truncation=True,
    use_fast=True,
)
    
# add LoRA layers on top of the quantized base model
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# specify epochs and learning rate
EPOCHS = 2
LEARNING_RATE = 2e-5

# set up training arguments
training_args = TrainingArguments(
    output_dir="sfft-trainer",
    overwrite_output_dir=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    optim="adafactor",
    seed=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    remove_unused_columns=False,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    logging_strategy="epoch", 
)

# set up trainer
trainer = SFTTrainer(
    model=model_t5_qn,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    peft_config=peft_config,
    dataset_text_field="summary",
    tokenizer=tokenizer,
    dataset_batch_size=5,
    max_seq_length=512,
    args=training_args,
)

# run trainer
trainer.train()

# specify where to save the pre-trained (domain adapted) SFT-model
trainer.model.save_pretrained("sft-domain-pretrained")

{'loss': 0.2407, 'learning_rate': 1e-05, 'epoch': 0.99}
{'loss': 0.2244, 'learning_rate': 0.0, 'epoch': 1.97}
{'train_runtime': 151.7069, 'train_samples_per_second': 1.977, 'train_steps_per_second': 0.488, 'train_loss': 0.23253621281804265, 'epoch': 1.97}


## 5.3. Update the model using DPO

In [60]:
from trl import DPOTrainer, create_reference_model
from peft import PeftModelForCausalLM

# load domain adapted SFT model
base_model = T5ForConditionalGeneration.from_pretrained(
    "sft-domain-pretrained",  
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32,
    device_map={"": 0},
)

# instantiate a PEFT model from a pretrained model and loaded PEFT weights.
model = PeftModelForCausalLM.from_pretrained(model=base_model, model_id="adapters", is_trainable=True)

# create reference model
model_ref = create_reference_model(model)

EPOCHS = 4
LEARNING_RATE = 2e-4

dpo_training_args = TrainingArguments(
    output_dir="dpo-model",
    remove_unused_columns=False,
    overwrite_output_dir=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    optim="adafactor",
    gradient_accumulation_steps=4,
    per_device_train_batch_size=4,
    logging_strategy="epoch",
)

dpo_trainer = DPOTrainer(
    model,  # base model from SFT pipeline
    model_ref,  # a copy of the SFT trained base model
    beta=0.1,  # temperature hyperparameter of DPO
    train_dataset=dpo_ds,  # dataset prepared above
    tokenizer=tokenizer,  # tokenizer
    args=dpo_training_args,  # training arguments e.g. batch size, lr, etc.
    max_length=150,
    max_prompt_length=300,
    max_target_length=128,
)

# train dpo model
dpo_trainer.train()

# specify where to save the DPO model
dpo_trainer.model.save_pretrained("trained-dpo")

{'loss': 0.6435, 'learning_rate': 0.00015000000000000001, 'rewards/chosen': -0.4232339859008789, 'rewards/rejected': -0.700904130935669, 'rewards/accuracies': 0.6349999904632568, 'rewards/margins': 0.2776701748371124, 'logps/rejected': -338.9629821777344, 'logps/chosen': -287.001953125, 'logits/rejected': -15.200387001037598, 'logits/chosen': -15.701244354248047, 'epoch': 0.96}
{'loss': 0.3177, 'learning_rate': 9.583333333333334e-05, 'rewards/chosen': -1.457634687423706, 'rewards/rejected': -3.256693124771118, 'rewards/accuracies': 0.8199999928474426, 'rewards/margins': 1.799058198928833, 'logps/rejected': -364.5209045410156, 'logps/chosen': -297.3459777832031, 'logits/rejected': -15.282553672790527, 'logits/chosen': -15.725358963012695, 'epoch': 2.0}
{'loss': 0.3075, 'learning_rate': 4.5833333333333334e-05, 'rewards/chosen': -2.101255178451538, 'rewards/rejected': -5.011992931365967, 'rewards/accuracies': 0.8349999785423279, 'rewards/margins': 2.910737991333008, 'logps/rejected': -382

## 5.4. Use the DPO model

In [61]:
# enable inference
dpo_trainer.model = dpo_trainer.model.merge_and_unload()
dpo_trainer.model.config.use_cache = True

Create new summaries with DPO model.

In [63]:
from utils.model_utils import _generate_summary

def _add_detoxified_summaries(sample, model, tokenizer):
    """
    Function to add summaries with DPO model.
    """
    
    # update embeddings in T5 model to 
    sample["dpo_summary"] = _generate_summary(sample["dialogue"], model, tokenizer)

    return sample


# use partial to pass the arguments to the map function
summaries_dataset_dpo = movie_dataset.map(partial(_add_detoxified_summaries, model=dpo_trainer.model, tokenizer=tokenizer), batched=False)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [64]:
from utils.eval_utils import _add_toxicty_column

summaries_dataset_dpo = _add_toxicty_column(summaries_dataset_dpo, "dpo_summary")
summaries_dataset = _add_toxicty_column(summaries_dataset, "summary")


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

<div class="alert alert-block alert-warning">
<b>Exercise 7</b>: Compare summaries from the DPO model to the reference model.
</div>

In [ ]:
##### complete your code here #####


###################################

## 5.5. Compare toxicity

In [65]:
import numpy as np

print(np.mean(summaries_dataset["toxicity_score"]), np.std(summaries_dataset["toxicity_score"]))
print(np.mean(summaries_dataset_dpo["toxicity_score"]), np.std(summaries_dataset_dpo["toxicity_score"]))

0.2025899624430167 0.31525101796310845
0.04223141155438498 0.14729100956793917


# Thank you!